In [1]:
import sys
from pathlib import Path

pyswap_path = Path("/home/zawadzkim/projects/pySWAP")
sys.path.append(str(pyswap_path.resolve()))


## Create a database connection

Storing data in the database instead instead of individual files would make it easier to manage multiple models and their versions.

In [2]:
from pyswap.base.database import connection as con

db_path = Path(pyswap_path, "pyswapdb.sqlite")
con.DatabaseConnection(db_path)


Database exists. Connecting...
Table swap_models exists in the database.
Table model_iterations exists in the database.
Table model_outputs exists in the database.


## Metadata

In [3]:
from pyswap.base.metadata import Metadata

meta = Metadata(author="John Doe",
                institution="University of Somewhere",
                email="john.doe@somewhere.com",
                project_name="Test",
                swap_ver="4.0")

## Simulation settings

In [4]:
from pyswap.base.simsettings import *
from datetime import date as dt

environment = Environment()
simulation_period = SimPeriod(tstart=dt(2018, 1, 1), 
                              tend=dt(2018, 1, 2))
output_dates = OutputDates(outdat=[dt(2018, 1, 1), dt(2018, 1, 1)])
output_files = OutputFiles(inlist_csv=['rain', 'evap'])

simset = SimSettings(environment=environment,
                     simulation_period=simulation_period,
                    output_dates=output_dates,
                    output_files=output_files)

## Meteorology

In [5]:
from pyswap.atmosphere.meteorology import MeteorologicalData

# Getting meteo data from KNMI
meteo = MeteorologicalData(metadata=meta)
meteo.weather_kmni()

In [6]:
from pyswap.atmosphere.meteorology import Meteorology

rainflux = {'time': [1.0, 360.0], 
            'rainflux': [20.0, 20.0]}


meteo = Meteorology(metfil='377',
                    meteo_data=meteo,
                    lat=43.0,
                    swetr=True,
                    swdivide=False,
                    swetsine=False,
                    swrain=0,
                    rainflux=rainflux)



## Crop

In [7]:
from pyswap.plant.crop import Crop

# Would be nice to have a subclass for managing the crop files

croprotation = {'cropstart':[dt(2018, 1, 1), dt(2018, 1, 1), dt(2018, 1, 1)],
                'cropend':[dt(2018, 1, 1), dt(2018, 1, 1), dt(2018, 1, 1)],
                'cropfile':['maizes', 'potatod', 'grassd'],
                'croptype':[1, 2, 3]}


crop = Crop(swcrop=True,
            rds=200.0,
            croprotation=croprotation)

## Irrigation

In [8]:
from pyswap.soilwater.irrigation import Irrigation

irrigation = Irrigation(schedule=False,
                        swirfix=False)  # is this is set to true, then an error is returned if additional parameters are not given

## Drainage

In [ ]:
from pyswap.soilwater.drainage import Drainage

drainage = Drainage(swdraintype=1)

## Soil water

In [9]:
from pyswap.soilwater.soilmoisture import SoilMoisture
from pyswap.soilwater.surfaceflow import SurfaceFlow
from pyswap.soilwater.evaporation import Evaporation


soilmoisture = SoilMoisture(swinco=2,
                            gwli=-75.0)

surfaceflow = SurfaceFlow(swpondmx=False,
                          pondmx=0.2,
                          swrunon=False)

evaporation = Evaporation(swcfbs=False,
                          cfevappond=1.25,
                          swredu=1,
                          cofredbl=0.35,
                          rsigni=0.5)


In [10]:
from pyswap.soilwater.soilprofile import SoilProfile
from pandas import DataFrame

df_soilprofile = DataFrame({
    'ISUBLAY': [1, 2, 3, 4],
    'ISOILLAY': [1, 1, 2, 2],
    'HSUBLAY': [10.0, 20.0, 30.0, 140.0],
    'HCOMP': [1.0, 5.0, 5.0, 10.0],
    'NCOMP': [10, 4, 6, 14]
})

soilprofile = SoilProfile(soilprofile=df_soilprofile,
                          swsophy=False,
                          swhyst=False,
                          swmacro=False)


In [11]:
from pyswap.soilwater.snow import SnowAndFrost

snow = SnowAndFrost(swsnow=False,
                    swfrost=False)


In [12]:
from pyswap.soilwater.richards import RichardsSettings

richards = RichardsSettings(swkmean=1,
                            swkimpl=False)


In [13]:
from pyswap.base.boundary import LateralDrainage, BottomBoundary

lateral_drainage = LateralDrainage(swdra=0)
bottom_boundary = BottomBoundary(swbbcfile=False, 
                                 swbotb=6)

## Putting the model together

In [14]:
from pyswap.base.model import Model

model = Model(metadata=meta,
              simsettings=simset,
              meteorology=meteo,
              crop=crop,
              irrigation=irrigation,
              soilmoisture=soilmoisture,
              surfaceflow=surfaceflow,
              evaporation=evaporation,
              soilprofile=soilprofile,
              snow=snow,
              richards=richards,
              lateral_drainage=lateral_drainage,
              bottom_boundary=bottom_boundary)

ModuleNotFoundError: No module named 'pyswap.soilwater.utils'